# Building an intelligent LLM Routing System using Langgraph

This notebook demonstrates how to build an LLM workflow using [Langgraph](https://github.com/langchain-ai/langgraph) for intelligent routing. 

## Use Case
Sales and marketing teams face diverse processing needs—from creative content generation to data-driven analysis to customer communications. Each of these tasks benefits from models with different strengths. Rather than using a one-size-fits-all approach, our routing system will analyze incoming requests, categorize them, and direct them to specialized models that excel at specific tasks.

Using langgraph, we'll be enable an LLM workflow that perform the following:

- Classify incoming requests by type and intent
- Create a decision framework that maps request categories to appropriate models
- Implement routing logic that directs requests to the optimal model

The following diagram depicts the overall architecture and the worklfow:

<img src="../../imgs/lab3-prompt-routing-architecture.png" width="800"> 

## Core Dependencies

We import the essential LangGraph and LangChain components for building our intelligent routing workflow:
- `entrypoint`, `task`: LangGraph decorators for defining workflow steps
- `BaseModel`: Pydantic for structured data validation
- `ChatBedrockConverse`: AWS Bedrock integration for LLM interactions

In [13]:
from typing_extensions import Literal
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_aws import ChatBedrockConverse
import boto3
from langgraph.func import entrypoint, task

## Environment and AWS Configuration

Setting up the AWS session and defining the specific models we'll use for different tasks. Each model is optimized for its specific role in the routing workflow.

In [59]:
session = boto3.session.Session()
region = session.region_name
bedrock_client = boto3.client('bedrock-runtime', region)

router_model_id = "us.amazon.nova-micro-v1:0"
strategy_model_id = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
social_media_model_id = "us.amazon.nova-lite-v1:0"


## Routing Schema Definition

This Pydantic model defines the structured output format for our routing logic, ensuring consistent and predictable routing decisions.

In [60]:
# Schema for structured output to use as routing logic
class Route(BaseModel):
    step: Literal["strategy", "social_media"] = Field(
        None, description="The` next step in the routing process"
    )


## Router LLM Setup

The router uses a lightweight, fast model (Nova Micro) optimized for classification tasks. Low temperature ensures consistent routing decisions.

In [61]:
router_llm = ChatBedrockConverse(
    model=router_model_id,  # or another Claude model
    temperature=0.1,
    max_tokens=None,
    client=bedrock_client,
)

In [62]:
# Augment the LLM with schema for structured output
router = router_llm.with_structured_output(Route)

## Strategy LLM Configuration

For strategy tasks, we use Claude 3.5 Sonnet - a more powerful model optimized for complex reasoning, analysis, and strategic thinking. Higher temperature allows for more creative strategic responses.

In [63]:
strategy_llm = ChatBedrockConverse(
    model=strategy_model_id,  # or another Claude model
    temperature=1.0,
    max_tokens=None,
    client=bedrock_client,
)

## Strategy Task Implementation

This task function handles complex marketing strategy requests, providing comprehensive analysis and actionable recommendations using the specialized strategy model.

In [64]:
@task
def llm_call_strategy(user_query: str):
    """A marketing strategy expert in Media & Entertainment that develops comprehensive strategy for the given request. """
    prompt_template = "You are a marketing strategy expert for Media & Entertainment companies. Please develop a comprehensive strategic response to this request:{{user_query}}. Include actionable insights on audience targeting, campaign planning, and ROI measurement."
    prompt = prompt_template.replace("{{user_query}}", user_query)
    result = strategy_llm.invoke(prompt)
    return result.content

## Social Media LLM Configuration

For social media content creation, we use Nova Lite - a model optimized for creative, engaging content generation while being cost-effective for high-volume social media tasks.

In [65]:
social_media_llm = ChatBedrockConverse(
    model=social_media_model_id,  # or another Claude model
    temperature=1.0,
    max_tokens=None,
    client=bedrock_client,
)

## Social Media Task Implementation

This task specializes in creating engaging, platform-specific social media content with the creative flair needed for viral marketing campaigns.

In [66]:
@task
def llm_call_social_media(user_query: str):
    """A social media expert for Media & Entertainment that create engaging social media content."""
    prompt_template = """You are a social media expert for Media & Entertainment companies. Create engaging social media content for this request:{{user_query}}. Include creative captions, relevant hashtags, and platform-specific tips."""
    prompt = prompt_template.replace("{{user_query}}", user_query)
    result = social_media_llm.invoke(prompt)
    return result.content

## Router Logic Implementation

The core routing function that analyzes incoming requests and directs them to the appropriate specialized handler based on content type and intent.

In [67]:
llm_router_system_prompt = """You are a specialized routing assistant for a Media & Entertainment marketing team. Your job is to classify incoming marketing requests into one of these categories: 1. STRATEGY - Questions about developing comprehensive marketing strategies, campaign planning, narratives, audience targeting, marketing analytics, ROI assessment, or long-term marketing plans 2. SOCIAL_MEDIA - Requests for creating social media posts, captions, hashtags, or content for specific platforms. Only respond with exactly one category label: either "strategy" or "social_media"."""
def llm_call_router(input_: str):
    """Route the input to the appropriate node"""
    # Run the augmented LLM with structured output to serve as routing logic
    decision = router.invoke(
        [
            SystemMessage(
                content=llm_router_system_prompt
            ),
            HumanMessage(content=input_),
        ]
    )
    return decision.step

## Workflow Orchestration

The main workflow entry point that coordinates the routing decision and task execution. LangGraph's `@entrypoint` decorator makes this the main execution point for our routing system.

In [68]:
# Create workflow
@entrypoint()
def router_workflow(input_: str):
    next_step = llm_call_router(input_)
    if next_step == "strategy":
        llm_call = llm_call_strategy
    elif next_step == "social_media":
        llm_call = llm_call_social_media

    return llm_call(input_).result()

## Testing Framework

This utility function enables us to observe the workflow execution in real-time, showing how requests flow through the routing system and which specialized handlers are activated.

In [72]:
def invoke(query):
    for step in router_workflow.stream(query, stream_mode="updates"):
        print(step)
        print("\n")

## Comprehensive Testing

We test the routing system with diverse marketing scenarios to validate that it correctly classifies and routes different types of requests to their optimal handlers.

In [ ]:
# Invoke
queries = [ 
    """We need to analyze the ROI of our summer blockbuster campaign across different marketing channels. Can you develop a framework for measuring effectiveness and determining which platforms delivered the best returns?""",
    """Our new documentary series appeals to both environmental activists and mainstream viewers. How should we develop a segmented marketing strategy that effectively targets these different audience groups while maximizing our limited budget?""",
    """We're repositioning our streaming service to compete with larger players. Can you outline a 12-month marketing strategy that builds brand recognition and emphasizes our unique content library to drive subscriber growth?""",
    """We need 5 Instagram post ideas for the premiere of our new sci-fi series "Nexus Point." Please include caption text and hashtag suggestions that will appeal to genre fans.""",
    """Create a viral TikTok challenge concept for promoting our new teen comedy movie. Include sample captions, music suggestions, and a catchy hashtag that could encourage user participation."""
]   

for query in queries:    
    invoke(query)



## Key Advantages of This LangGraph Approach

- **Structured Routing**: Pydantic schemas ensure reliable routing decisions
- **Task Specialization**: Different models optimized for specific marketing functions  
- **Real-time Observability**: Stream mode provides visibility into workflow execution
- **Scalable Architecture**: Easy to add new categories and specialized handlers
- **Cost Optimization**: Uses appropriate model complexity for each task type